<a href="https://colab.research.google.com/github/vy-phung/Data-analysis/blob/main/Basic_language_translator_and_speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is just a basic language translator, which translates from one language to another language and a speech to text, which converts a speech to a text.   

1. For speech to text, I will download a standup comedy speech on Youtube ('How The British Took Over India' - TREVOR NOAH) https://youtu.be/QhMO5SSmiaA
2. For language translator, I will use 3 options to translate: 
*   Typing (You have to typing directly) 
*   A file (a file you've already had in your path) (in this case I use *.txt)
*   Using a text from the above speech to text section

About the APIKEY and URL for both Language translator and Speech to text, I take them from my IBM Watson Studio account. Actually, I feel ok if you intend to use this APIKEY or URL for the purpose of learning. 






**AUTHENTICATE**

In [ ]:
! pip install --upgrade "ibm-watson>=4.2.1"  

In [2]:
from ibm_watson import LanguageTranslatorV3, SpeechToTextV1 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [3]:
ltapikey = "p9nTUgusELZe1dNWLQwDjbg2EAzqlQJNYF-XyW608xeF"
lturl = "https://api.us-south.language-translator.watson.cloud.ibm.com/instances/df06279e-fc78-4863-aeef-a809865bfdfe"
sttapikey = "O43QI28AeQ-qtV9hdsvj8QVSAUX1q8bhoh3Xf5Vr3pPr"
stturl = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/7252e247-f575-456d-a86e-431c2622d39e"

In [4]:
# Language Translator
ltauthenticator = IAMAuthenticator(ltapikey)
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=ltauthenticator
)

language_translator.set_service_url(lturl)
# Speech To Text
sttauthenticator = IAMAuthenticator(sttapikey)
SpeechToText = SpeechToTextV1(
    authenticator=sttauthenticator
)

SpeechToText.set_service_url(stturl)


**Speech to Text**

In this case, because I use audio/mp3 to run SpeechToText, I will have to convert the youtube speech to mp3 file.

I use the code and instruction from "https://github.com/silvanohirtie/youtube-mp3-downloader.git" to convert. 

In [6]:
pip install youtube-dl

     |████████████████████████████████| 1.9 MB 5.1 MB/s 


In [7]:
from __future__ import unicode_literals
import youtube_dl
print("Insert the link")
link = input ("")
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',
    }],
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([link])

Insert the link
https://youtu.be/QhMO5SSmiaA
[youtube] QhMO5SSmiaA: Downloading webpage
[download] Destination: 'How The British Took Over India' - TREVOR NOAH (from 'Afraid Of The Dark' on Netflix)-QhMO5SSmiaA.m4a
[download] 100% of 5.41MiB in 02:07
[ffmpeg] Correcting container in "'How The British Took Over India' - TREVOR NOAH (from 'Afraid Of The Dark' on Netflix)-QhMO5SSmiaA.m4a"
[ffmpeg] Destination: 'How The British Took Over India' - TREVOR NOAH (from 'Afraid Of The Dark' on Netflix)-QhMO5SSmiaA.mp3
Deleting original file 'How The British Took Over India' - TREVOR NOAH (from 'Afraid Of The Dark' on Netflix)-QhMO5SSmiaA.m4a (pass -k to keep)


In [ ]:
print('Drop the file path')
file = input("")
with open(file, 'rb') as f:
  res = SpeechToText.recognize(audio=f,content_type='audio/mp3', model='en-US_NarrowbandModel').get_result()

Drop the file path
/content/'How The British Took Over India' - TREVOR NOAH (from 'Afraid Of The Dark' on Netflix)-QhMO5SSmiaA.mp3


In [ ]:
res

**Language Translator**

In [9]:
import pandas as pd
import numpy as np

In [10]:
language_list = language_translator.list_identifiable_languages().get_result()
language = language_list['languages']
language = pd.DataFrame(language)
language = language.set_index("name")
language.index.names = [None]
language

,language
Afrikaans,af
Arabic,ar
Azerbaijani,az
Bashkir,ba
Belarusian,be
...,...
Ukrainian,uk
Urdu,ur
Vietnamese,vi
Simplified Chinese,zh


In [11]:
print('Insert your input language')
i = input("")
i = language.loc[i,:]
print("Insert your output langugae")
o = input("")
o = language.loc[o,:]
model_id = "%s-%s"%(i[0],o[0])
print("There are 3 choices for translation:\n",
      "1. Typing a content\n",
      "2. Drop a path file you have\n",
      "3. Translate from voice text\n",
      "Type the number 1 or 2 or 3 to choose")
content = input("")
if content == "1":
  text_to_translate = input("Your content you want translate here")
  text = '%s'%text_to_translate
elif content == "2":
  text = input("Drop a path of your file")
  t = '%s'%text
  with open(t) as f:
    text = f.read()
elif content == "3":
  t = []
  for f in res['results']:
    texts = f['alternatives'][0]['transcript']
    t.append(texts)
  text = ' '.join(t)
# Translate
translation = language_translator.translate(
    text=text,
    model_id=model_id).get_result()
tran = translation['translations'][0]['translation']
print(tran)

Insert your input language
English
Insert your output langugae
Vietnamese
There are 3 choices for translation:
 1. Typing a content
 2. Drop a path file you have
 3. Translate from voice text
 Type the number 1 or 2 or 3 to choose
2
Drop a path of your file/content/test.txt
Xin chào, tên tôi là Vy.
